# 多文件加载示例

演示如何使用 DataSession 加载多个 Parquet 文件

## 三种场景

1. **同构文件合并** - 结构相同的多个文件（concat）
2. **异构文件关联** - 有外键关系的多个文件（join）
3. **独立文件批量加载** - 多个不相关的文件


In [ ]:
# 初始化
import polars as pl
from src.session import DataSession

session = DataSession()

## 场景 1：同构文件合并 (Concat)

**适用于**：多年数据、分片数据、分批次导出的数据


In [ ]:
# 示例 1.1: 合并多个指定文件
session.load_multiple_concat(
    ['data/2022.parquet', 'data/2023.parquet', 'data/2024.parquet'],
    alias='all_years'
)

# 使用合并后的数据
all_years_df.group_by('业务年度').agg([
    pl.col('总保费').sum(),
    pl.len().alias('保单数')
])

In [ ]:
# 示例 1.2: 使用 glob 模式
session.load_multiple_concat(
    ['data/year_*.parquet'],  # 匹配所有 year_开头的文件
    alias='all_data'
)

print(f"总数据量: {all_data_df.height:,} 行")

In [ ]:
# 示例 1.3: 容错模式（schema 不完全一致）
session.load_multiple_concat(
    ['old_format/*.parquet', 'new_format/*.parquet'],
    alias='mixed',
    ignore_schema_errors=True  # 缺失字段会填充 null
)

## 场景 2：异构文件关联 (Join)

**适用于**：主表-维度表、订单-客户-产品等有外键关系的数据


In [ ]:
# 示例 2.1: 简单的两表 join
session.load_multiple_join(
    files={
        'policy': 'data/保单明细.parquet',
        'customer': 'data/客户信息.parquet'
    },
    joins=[
        {
            'left': 'policy',
            'right': 'customer',
            'on': '客户ID',
            'how': 'left'
        }
    ],
    result_alias='enriched_policy'
)

# 现在可以访问客户信息
enriched_policy_df.select(['保单号', '客户名称', '客户类型', '总保费'])

In [ ]:
# 示例 2.2: 多表连续 join
session.load_multiple_join(
    files={
        'policy': 'data/保单.parquet',
        'customer': 'data/客户.parquet',
        'product': 'data/产品.parquet',
        'agent': 'data/代理人.parquet'
    },
    joins=[
        # 第一步：关联客户
        {'left': 'policy', 'right': 'customer', 'on': '客户ID', 'how': 'left'},
        # 第二步：关联产品（left 会自动使用上一步结果）
        {'left': 'policy', 'right': 'product', 'on': '产品代码', 'how': 'left'},
        # 第三步：关联代理人
        {'left': 'policy', 'right': 'agent', 'on': '代理人代码', 'how': 'left'}
    ],
    result_alias='full_data'
)

# 完整的宽表
full_data_df.columns  # 包含所有表的字段

In [ ]:
# 示例 2.3: 复杂 join（多字段、内连接）
session.load_multiple_join(
    files={
        'orders': 'orders.parquet',
        'items': 'order_items.parquet'
    },
    joins=[
        {
            'left': 'orders',
            'right': 'items',
            'on': ['订单号', '年份'],  # 多字段 join
            'how': 'inner',
            'suffix': '_item'  # 重名字段后缀
        }
    ],
    result_alias='order_details'
)

## 场景 3：独立文件批量加载

**适用于**：多个不相关的数据集，需要分别分析


In [ ]:
# 示例 3: 批量加载独立数据集
session.load_multiple_independent({
    'sales': 'data/销售数据.parquet',
    'hr': 'data/人力资源.parquet',
    'finance': 'data/财务数据.parquet',
    'inventory': 'data/库存.parquet'
})

# 分别使用
print("销售数据:", sales_df.height, "行")
print("人力资源:", hr_df.height, "行")
print("财务数据:", finance_df.height, "行")
print("库存数据:", inventory_df.height, "行")

## 查看会话状态


In [ ]:
# 查看所有已加载的数据
session.summary()

In [ ]:
# 生成 AI Context
print(session.get_ai_context())

## 实际案例：保险数据分析


In [ ]:
# 案例：分析 2020-2024 年保费趋势，并关联产品和客户信息

# Step 1: 合并多年保单数据
session.load_multiple_concat(
    ['data/policy_{2020..2024}.parquet'],
    alias='all_policies'
)

# Step 2: 关联维度表
session.load_multiple_join(
    files={
        'policy': 'df_all_policies',  # 可以引用已加载的
        'customer': 'data/customer_master.parquet',
        'product': 'data/product_master.parquet'
    },
    joins=[
        {'left': 'policy', 'right': 'customer', 'on': '客户ID', 'how': 'left'},
        {'left': 'policy', 'right': 'product', 'on': '产品代码', 'how': 'left'}
    ],
    result_alias='analysis_ready'
)

# Step 3: 分析
result = analysis_ready_df.group_by(['业务年度', '产品类型', '客户类型']).agg([
    pl.col('总保费').sum(),
    pl.len().alias('保单数')
]).sort('业务年度', '总保费', descending=True)

print(result)